### 欠損値補間の自動化

In [3]:
def impute(df, strategy='mean'):
    from sklearn.preprocessing import Imputer
    imp_num = Imputer(missing_values='NaN', strategy=strategy, axis=0)
    # 欠損がある列を取り出す。
    isnull_sums = df.isnull().sum() #欠損値の数を計算
    isnull_index = []
    for index, isnull_sum in enumerate(isnull_sums): #欠損値の数が0より多い場合のインデックスを抽出
        if isnull_sum > 0:
            isnull_index.append(index)
            isnull_df = df.iloc[:, isnull_index]

    display('欠損値があるデータの抽出',isnull_df)
    print('-------------------------------')
    display('データ型の確認',isnull_df.dtypes)
    print('-------------------------------')

    # 数字データのみの列を取り出す
    df_nums = df.iloc[:, isnull_index].select_dtypes(include=[int, float])
    df_not_nums = df.iloc[:, isnull_index].select_dtypes(exclude=[int, float])

    if not df_nums.empty:
        display('以下のデータを補間します。',df_nums)
        print('-------------------------------')

        # インスタンス化した imp_num に与えて、欠損値を補間
        imputed_data = imp_num.fit_transform(df_nums)
        print(imputed_data)
        df_nums[:] = imputed_data
        display('以下のデータで補間します。',df_nums)
        print('-------------------------------')

        # 元のdfに代入
        df[df_nums.columns] = df_nums
    display('補間完了',df)
    print('-------------------------------')

    if not df_not_nums.empty:
        display('以下は数値データでは無いので欠損値の補間が出来ません。',df_not_nums)

### カテゴリカル・データ補間の自動化

In [1]:
# カテゴリカル・データ補間の自動化
def categorical(data, size2int, data_column):
    data[data_column] = data[data_column].map(size2int)
    display(data)

### ダミー化の自動化

In [2]:
# ダミー化の自動化
def dummy(data, data_column, categories):
    # size の列のデータタイプを category に変換
    data[data_column] = pandas.Categorical(data[data_column], categories=categories, ordered=True)

    # get_dummies()関数を利用してダミー化,dummy_na=Trueの引数を与える。
    data = pandas.get_dummies(data, columns=[data_column], dummy_na=True)

    display(data)

### 特徴量選択の自動化

In [4]:
def importance(fit_data, columns, count):
    '''
    fit_dataはランダムフォレストか決定木のfit
    columnsは特徴量のラベールネーム
    countは抽出する特徴量の数かmean（平均以上）
    '''
    import numpy
    from matplotlib import pyplot
    import japanize_matplotlib

    feature_names = numpy.array(columns)
    # 特徴重要度はfeature_importance_に格納されている
    feature_importances = fit_data.feature_importances_
    # ソートするが、返す値はソート前のindex、これの通りに後から並び替えたりできる
    indices = numpy.argsort(feature_importances)

    pyplot.figure(figsize=(11, 7))
    pyplot.title('Feature Imoportances')

    # barhで横向きの棒グラフの描写設定（縦はbar）,range(len(indices)):横軸9個, feature_importances[indices]:indecesの順番でプロット
    print(range(len(indices)))
    print(feature_importances[indices])
    pyplot.barh(range(len(indices)), feature_importances[indices], color='b', align='center')
    pyplot.yticks(range(len(indices)), feature_names[indices])
    pyplot.show()

    # 特徴量の重要度から決定領域をプロットするためのデータを決める
    print(feature_names[indices])
    if count == 'mean':
        mu = feature_importances.mean()
        top_culumns_name = feature_names[indices][::-1][feature_importances[indices][::-1] > mu]
        print(top_culumns_name)
        return top_columns_name
    elif isinstance(count, int):
        # [::-1]で逆順に、[:2]で最初から二番目まで取り出し
        top_columns_name = feature_names[indices][::-1][:count]
        print(top_columns_name)
        return top_columns_name